In [7]:
!pip install langchain pypdf chromadb openai tiktoken unstructured requests

In [15]:
import os

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

os.environ['OPENAI_API_KEY'] = ''

file_path = "./sample_data/UniData.xlsx"
loader = UnstructuredExcelLoader(file_path)
pages = loader.load()

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectordb = Chroma.from_documents(pages, embedding=embeddings,
                                 persist_directory="./vdb")
vectordb.persist()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
pdf_qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0) ,
                                               vectordb.as_retriever(), memory=memory,
                                               condense_question_llm = OpenAI(temperature=0, model='gpt-3.5-turbo'),
                                               )

result = pdf_qa({"question": "What are the courses in finance?"})
print("Answer:")
result["answer"]

InvalidRequestError: ignored